# Data Analyst Associate Practical Exam Submission

### Company Background 
Vivendo is a fast food chain in Brazil with over 200 outlets. As with many fast food establishments, customers make claims against the company. For example, they blame Vivendo for suspected food poisoning. The legal team, who processes these claims, is currently split across four locations. The new head of the legal department wants to see if there are differences in the time it takes to close claims across the locations. 
### Customer Question 
The legal team has provided a data set where each row is a claim made against the company. They would like to have the answer to the following questions: 
* How does the number of claims differ across locations? 
* What is the distribution of time to close claims? 
* How does the average time to close claims differ by location?

### Importing Modules

In [1]:

# import libraries
import pandas as pd
import numpy as np 
import plotly.express as px
import plotly.offline as po
po.init_notebook_mode(connected=True)

### Loading and Assessing Claims Table

In [2]:
# Loading data
claims = pd.read_csv('claims.csv')
# Table preview and shape
print(claims.shape)
claims.head()

(98, 8)


,Claim ID,Time to Close,Claim Amount,Amount Paid,Location,Individuals on Claim,Linked Cases,Cause
0,0000000-00.2010.0.00.0094,2082,"R$50,000.00",34297.47,RECIFE,2,False,NaN
1,0000000-00.2011.0.00.0086,3591,"R$180,000.00",56383.54,SAO LUIS,7,False,NaN
2,0000000-00.2012.0.00.0092,2997,"R$70,000.00",12366.76,SAO LUIS,3,False,vegetables
3,0000000-00.2013.0.00.0096,1143,"R$95,104.00",29217.16,FORTALEZA,4,False,NaN
4,0000000-00.2014.0.00.0097,117,"R$29,000.00",12610.50,FORTALEZA,1,False,meat


In [3]:
# Checking table information
claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Claim ID              98 non-null     object 
 1   Time to Close         98 non-null     int64  
 2   Claim Amount          98 non-null     object 
 3   Amount Paid           98 non-null     float64
 4   Location              98 non-null     object 
 5   Individuals on Claim  98 non-null     int64  
 6   Linked Cases          98 non-null     bool   
 7   Cause                 20 non-null     object 
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 5.6+ KB


In [4]:
# Checking for duplicates
claims.duplicated().sum()

0

In [5]:
# Checking for empty values
claims.isnull().sum()

Claim ID                 0
Time to Close            0
Claim Amount             0
Amount Paid              0
Location                 0
Individuals on Claim     0
Linked Cases             0
Cause                   78
dtype: int64

In [6]:
# Checking unique values of variables
print(len(claims['Claim ID'].unique()))
print(claims['Linked Cases'].unique())
print(claims['Location'].unique())
print(claims['Cause'].unique())

98
[False  True]
['RECIFE' 'SAO LUIS' 'FORTALEZA' 'NATAL']
[nan 'vegetables' 'meat']


### Data Issues
Tidiness issue
* Variable headers are capitalize and have spaces

Quality issue
* Unit of currency in the variable, `Claim Amount`.
* Missing values in the variable, `Cause`.

### Cleaning Data
#### Making copy of original table

In [7]:
# copy original data
claims_clean = claims.copy()

#### Tidiness
Issue #1: Variable headers are capitalize

**Define**: Variable headers should be lower case with underscore as space for easy accessing in python. Change variable headers to lower case and replace space with underscore.

**Code**

In [8]:
# Changing variable headers to lower case and replace space with underscore
claims_clean.columns = claims_clean.columns.str.replace(' ', '_').str.lower()

**Test**

In [9]:
# Print table columns
list(claims_clean.columns)

['claim_id',
 'time_to_close',
 'claim_amount',
 'amount_paid',
 'location',
 'individuals_on_claim',
 'linked_cases',
 'cause']

#### Quality
Issue #1: Unit of currency in the variable, `Claim Amount`. 

**Define**: To perform calculation on this variable, currency unit should be remove. Remove currency unit and convert to numeric.

**Code**

In [10]:
# Remove currency unit and convert to integer
claims_clean.claim_amount = np.int64(claims_clean.claim_amount.str.strip('R$').str.replace(',', '').astype(float))

**Test**

In [11]:
# Preview claim amount variable
claims_clean.claim_amount[:5]

0     50000
1    180000
2     70000
3     95104
4     29000
Name: claim_amount, dtype: int64

Issue #2: Missing values in the variable, `Cause`.

**Define**: There is no cause value is for over 50% of the observations. As requested, all missing value will be replaced with *unknown*

**Code**

In [12]:
# Fill all missing value in cause column with "unknown"
claims_clean.cause =  claims_clean.cause.fillna('unknown')

**Test**

In [13]:
# Check unique values in cause column
print(claims_clean.cause.unique())

['unknown' 'vegetables' 'meat']


In [14]:
# Preview cleaned data
claims_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   claim_id              98 non-null     object 
 1   time_to_close         98 non-null     int64  
 2   claim_amount          98 non-null     int64  
 3   amount_paid           98 non-null     float64
 4   location              98 non-null     object 
 5   individuals_on_claim  98 non-null     int64  
 6   linked_cases          98 non-null     bool   
 7   cause                 98 non-null     object 
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 5.6+ KB


## Data Validation
Describe the validation tasks you performed and what you found. Have you made any changes to the data to enable further analysis? Remember to describe what you did for every column in the data. 

_**Description**_

There are 98 rows with 8 columns in the data. While assessing the data no duplicates was found but there were missing values in `cause` column only. This missing value was more than 50% percent of the total record. The below changes was made for each columns:
 
1. There are 98 unique `Claim ID` as expected.
2. `Time to Close` was numeric, only the header was changed.
3. The currency unit and comma in `Claim Amount` was removed. This column was then converted to numeric.
4. For `Amount Paid` column, only the header was changed.
5. `Location` column contained 4 unique variable as expected.
6. For `Individuals on Claim` column, only the header was changed.
7. `Linked Cases` column contained only True and False as expected.
8. Missing values in `Cause` column was replaced with **unknown**.

**NOTE**: All column headers was changed to lower case and spaces replaced with underscore for easy accessing.

## Data Discovery and Visualization
Describe what you found in the analysis and how the visualizations answer the customer questions in the project brief. In your description you should:
- Include at least two different data visualizations to demonstrate the characteristics of single variables
- Include at least one data visualization to demonstrate the relationship between two or more variables
- Describe how your analysis has answered the business questions in the project brief

***Description***




### 1. How does the number of claims differ across locations? 

From our data, the claims are coming from 4 unique location. The distribution belows shows most of the claims comes from Sao Luis followed by Recife location, while Fortaleza is higher than Natal with 1 claim.

In [15]:
fig = px.histogram(claims_clean, x="location",
                   title='Distribution of claims across locations', 
                   text_auto= True,
                   width=600,
                   labels={'location':'Location', 'count':'Count of claims'})
fig.update_traces(textposition='outside', cliponaxis=False)
fig.update_yaxes(showticklabels=False)
fig.show()

We've seen the number of claims in each location. Below is a brief view of claims that are linked to other cases for each location. The plot shows that all claims in Fortaleza is not linked to other cases, while in other location there is at least one claim that is linked to other cases.

In [16]:
fig = px.histogram(claims_clean, x="location", pattern_shape='linked_cases',
                   title='Distribution of linked claims across locations', 
                   text_auto= True,
                   width=600,
                   labels={'location':'Location'})
fig.update_traces(textposition='inside', cliponaxis=False)
fig.update_yaxes(showticklabels=False, title='Claim count')
fig.show()

### 2. What is the distribution of time to close claims? 
From the histogram below, we can see that there is high frequency of claims for time to close range of 200 to 600. **This means most claims usually take 200 to 600 days before it close**. There also seem to be an outlier in this variable and the boxplot at the margin shows that. This could be claims that is linked to other cases that is why it took that long before it was closed. Further analysis can be done to confirm this inference. 

In [17]:
fig = px.histogram(claims_clean, x="time_to_close", nbins=20, marginal="box",
                   title='Distribution of time to close claims',
                   width=600, labels={'time_to_close': 'Time to close'})
fig.show()

### 3. How does the average time to close claims differ by location?
**Sao Luis** has the highest average time to close (**approx. 1000 days**) followed by **Recife** (**approx. 860 days**). The bar chart also shows that, on average, it takes **approximately 580 days** to close a claim in **Natal** which is the lowest average time to close amongst the four locations while it takes **approximately 780 days** on average to close a case in **Fortaleza**.

In [18]:
avg  = claims_clean.groupby('location')['time_to_close'].mean()
fig = px.bar(avg, x= avg.index, y= avg.values,
             title='Average time to close across locations', 
                   text_auto= '.1s',
                   width=600,
                   labels={'location':'Location', 'y':'Avg. Time to close'})
fig.update_traces(textposition='outside', cliponaxis=False)
fig.update_yaxes(showticklabels=False)
fig.show()